In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.insert(0, '..')
import fashion.preprocessing as prep
# display all columns
pd.set_option('display.max_columns', None)

# show all results of the notebook not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
shops = prep.load_shops('../data/20200120_filiali.csv')
shops.head()

In [ ]:
import importlib
importlib.reload(prep)

sales17 = prep.load_sales('../data/20200120_sales17.csv', shops)
sales17.head()
len(sales17)

In [ ]:
sales17.dtypes

In [ ]:
importlib.reload(prep)

prods = prep.load_products('../data/20200120_barcode.csv')
prods.head()

In [ ]:
which = 'ProductID'
len(prods)
prods[which].value_counts().head(20)
prods[which].unique()
prods[(prods[which] == 67527) ]#.ColorDescription.unique()

In [ ]:

def explore(df):
    
    # look at values for each column
    for col in df.columns:
        
        print(col)
        uniques = df[col].unique()
        print(len(uniques))
        if len(uniques) > 100:
            continue
        counts = {u:np.sum(df.eval('{0}=="{1}"'.format(col, u))) for u in uniques}
        
        fig, ax = plt.subplots(figsize=(10,10))
        ax.bar(x=range(len(uniques)), height=[counts[u] for u in uniques], tick_label=uniques)
        ax.set_title(col)
        plt.show()
        

explore(barcode)
        


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
_ = ax.hist(sales17.Net_Income, bins=100)
ax.set_yscale('log')
plt.show()

In [ ]:
sales1819 = prep.load_sales('../data/20200120_sales1819.csv', shops)
sales1819.head()
len(sales1819)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
_ = ax.hist(sales1819.Net_Income, bins=100)#, range=(-1000, 1000))
ax.set_yscale('log')
plt.show()

In [ ]:
#sales1819[abs(sales1819.Net_Income) > 100]
sales1819[sales1819.Volume > 10]

In [ ]:
len(shops)

In [ ]:
sales1819.head(10)
year = sales1819.Date

In [ ]:
test = pd.to_datetime(year.astype(str))
pd.DatetimeIndex(test).year.unique()

In [ ]:
minisales = sales17#.head(100000)
idx = pd.DatetimeIndex(minisales.Date.astype(str))
years = idx.year
months = idx.month
days = idx.day

#minisales18 = minisales[pd.DatetimeIndex(minisales.Date.astype(str)).year == 2018]
#years = pd.DatetimeIndex(minisales18.Date.astype(str)).year

years.value_counts()

In [ ]:
#months[years==2016].value_counts()
#months[years==2017].value_counts()
#months[years==2018].value_counts()
#months[years==2019].value_counts()

vcounts = months[years==2016].value_counts()
vcounts

sales = [vcounts.loc[m] if m in vcounts else 0 for m in xs]
sales

fig, ax = plt.subplots()
xs = range(1, 13)
for y in [2016, 2017, 2018, 2019]:
    vcounts = months[years==y].value_counts()
    ax.plot(xs, [vcounts.loc[m] if m in vcounts else 0 for m in xs]/np.sum(vcounts), label='{} ({})'.format(y, np.sum(vcounts)))
ax.legend()
ax.set_xlabel('month of the year')
ax.set_ylabel('fraction of yearly sales')
ax.set_title('sales17 data')
plt.show()



In [ ]:
minisales.head()
minisales.dtypes
minisales.memory_usage() / 10**9
np.sum(minisales.memory_usage() / 10**9)

In [ ]:
import networkx as nx


In [ ]:
G=nx.Graph()
G.add_node('w2')
G.add_node('w')
G.add_edge(1,2)
G.add_edge(1,2)
G.nodes()
G.edges()
G[1][2]['weight'] = 1
G[1][2]


In [ ]:
prods = prep.load_products('../data/20200120_barcode.csv')
prods.head()

In [ ]:
prods.Size.value_counts().head(50)

In [ ]:
S = nx.Graph()

# add nodes
for size in prods.Size.unique():
    S.add_node(size)
S.nodes()

# add edges
t0 = time.time()
for i, pid in enumerate(prods.ProductID.unique()):
    pid_sizes = prods[prods.ProductID == pid]['Size'].unique()
    if len(pid_sizes) <= 1:
        continue
    
    for s1, s2 in itertools.combinations(pid_sizes,2):
        # skip PZ
        if 'PZ' in [s1, s2]:
            continue
        
        # add a new edge
        if (s1, s2) not in S.edges():
            S.add_edge(s1, s2, weight=1)
        else:
            S[s1][s2]['weight'] += 1
    
    if i>10000:
        break
        
print((time.time()-t0)/60, 'minutes')

# edges = [e for e in sizes.edges()]
# edges[:5]
# widths = [sizes[s1][s2]['weight'] for s1, s2 in edges]
# widths[:5]

# fgi, ax = plt.subplots(figsize=(10,10))
# nx.draw_networkx_edges(sizes,
#                        pos=nx.spring_layout(sizes),
#                        width=np.array(widths)/50,
#                        ax=ax)
# lim = 0.2
# ax.set_xlim(-lim,lim)
# ax.set_ylim(-lim,lim)

# for c in nx.connected_components(sizes):
#     #print(c)
#     #subgraph = sizes.subgraph(c)
#     #widths = [sizes[s1][s2]['weight'] for s1, s2 in subgraph.edges()]
#     #fig, ax = plt.subplots(figsize=(5, 5))
#     nx.draw(sizes, with_labels=True)
#     #lim = 0.2
#     #ax.set_xlim(-lim,lim)
#     #ax.set_ylim(-lim,lim)
#     break

In [ ]:
help(nx.draw_networkx_edges)

In [ ]:

for i, sg in enumerate(nx.connected_components(S)):
    subgraph = S.subgraph(sg)
    fig, ax = plt.subplots(figsize=(50,50))
    pos = nx.spring_layout(subgraph)
    edgelist = subgraph.edges()
    #print(edgelist)
    widths = np.array([subgraph[n1][n2]['weight'] for n1, n2 in edgelist])
    nx.draw(subgraph, with_labels=True, pos=pos)
    nx.draw_networkx_edges(subgraph, pos=pos, width=widths/100)
    plt.savefig('network_{}.pdf'.format(i))
    
# for edge in S.edges:
#     print(edge)
#     n1 = edge[0]
#     n2 = edge[1]
#     print(S[n1][n2]['weight'])

In [ ]:
prods.Gender.unique()
prods[(prods.Gender == 'UO') | (prods.Gender == 'DO')].Size.unique()
prods[(prods.Gender == 'BA') | (prods.Gender == 'BO')].Size.unique()

In [ ]:
G = nx.Graph()
for city in ['Oxford', 'London', 'Cambridge']:
    G.add_node(city)

G.add_edge('Oxford', 'London')
G.add_edge('Oxford', 'Cambridge')
print(G.nodes)

fig, ax = plt.subplots(figsize=(10,10))
nx.draw(G, with_labels=True)
# lim = 0.2
# ax.set_xlim(-lim,lim)
# ax.set_ylim(-lim,lim)

In [ ]:
help(nx.draw_networkx)

In [ ]:

# loop over all the genders
for gender in prods.Gender.unique():
    
    # check if there is a reasonable number of items
    if type(gender) != str or prods.Gender.value_counts()[gender] < 100:
        continue
        
    print(gender)
    
    # create a subprods
    these_prods = prods[prods.Gender == gender]
    
    # create a graph with all the sizes
    G = nx.Graph()
    for size in these_prods.Size.unique():
        S.add_node(size)
    S.nodes()
    
    # add the edges
    for i, pid in enumerate(these_prods.ProductID.unique()):
        pid_sizes = these_prods[these_prods.ProductID == pid]['Size'].unique()
        if len(pid_sizes) <= 1:
            continue
    
        for s1, s2 in itertools.combinations(pid_sizes,2):
            # skip PZ
            if 'PZ' in [s1, s2]:
                continue

            # add a new edge
            if (s1, s2) not in S.edges():
                S.add_edge(s1, s2, weight=1)
            else:
                S[s1][s2]['weight'] += 1

        if i>1000:
            break
    
    S.edges()
    for i, sg in enumerate(nx.connected_components(S)):
        
        if len(sg) <= 1:
            continue
        
        # prepare
        fig, ax = plt.subplots(figsize=(10,10))
        subgraph = S.subgraph(sg)
        pos = nx.spring_layout(subgraph)
        edgelist = subgraph.edges()
        widths = np.array([subgraph[n1][n2]['weight'] for n1, n2 in edgelist])
        
        nx.draw(subgraph, with_labels=True, pos=pos)
        print(widths)
        print(np.log(widths))
        nx.draw_networkx_edges(subgraph, pos=pos, width=np.log(widths))
        ax.set_title('Gender = {}'.format(gender), size=50)
        plt.savefig('network_{}_{}.pdf'.format(gender, i))
    
    
prods.Gender.value_counts()

In [ ]:
type(np.nan)